<a href="https://colab.research.google.com/github/SabbirAdid/Fire-Prediction-Using-Drone-Image/blob/main/FD_With_Satellite_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Zip ফাইল আপলোড করুন
from google.colab import files
uploaded = files.upload()  # zip ফাইল আপলোড করুন (যেমন: Fire and Non-Fire satellite picture.zip)

# STEP 2: Zip ফাইল আনজিপ করুন
import zipfile
import os

zip_filename = next(iter(uploaded))  # আপলোডকৃত zip ফাইলের নাম বের করা
extract_path = "/content/fire_dataset"  # এক্সট্রাকশন ডিরেক্টরি

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ডেটাসেট সফলভাবে আনজিপ করা হয়েছে!")

# STEP 3: Fire/Non-Fire ফোল্ডার খুঁজে বের করা
inner_dirs = os.listdir(extract_path)
for item in inner_dirs:
    full_path = os.path.join(extract_path, item)
    if os.path.isdir(full_path) and 'Fire' in os.listdir(full_path):
        image_root_path = full_path
        break

print("✅ Fire/Non-Fire ফোল্ডার পাওয়া গেছে:", image_root_path)

# STEP 4: ইমেজ প্রসেসিং ও ব্যালান্সিং ফাংশন
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

image_size = (250, 250)  # রিসাইজ করা হবে 250x250

def load_and_balance_images(root_path, image_size=(250, 250)):
    class_data = {"Fire": [], "Non-Fire": []}
    image_names = {"Fire": [], "Non-Fire": []}

    for folder in ["Fire", "Non-Fire"]:
        folder_path = os.path.join(root_path, folder)
        for img_file in tqdm(os.listdir(folder_path), desc=f"Processing {folder}"):
            img_path = os.path.join(folder_path, img_file)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, image_size)
                img = img.astype('float32') / 255.0
                class_data[folder].append(img)
                image_names[folder].append(img_file)
            except Exception as e:
                print(f"❌ {img_path} পড়া যায়নি - {e}")

    # 🔄 ব্যালান্স: Fire এবং Non-Fire সমান সংখ্যার ছবি রাখার জন্য
    min_count = min(len(class_data["Fire"]), len(class_data["Non-Fire"]))
    fire_indices = random.sample(range(len(class_data["Fire"])), min_count)
    nonfire_indices = random.sample(range(len(class_data["Non-Fire"])), min_count)

    balanced_images = []
    balanced_labels = []
    balanced_names = []

    for idx in fire_indices:
        balanced_images.append(class_data["Fire"][idx])
        balanced_labels.append(1)  # Fire = 1
        balanced_names.append(image_names["Fire"][idx])

    for idx in nonfire_indices:
        balanced_images.append(class_data["Non-Fire"][idx])
        balanced_labels.append(0)  # Non-Fire = 0
        balanced_names.append(image_names["Non-Fire"][idx])

    print(f"✅ ব্যালান্সড ডেটাসেট: Fire = {min_count}, Non-Fire = {min_count}")
    return np.array(balanced_images), np.array(balanced_labels), balanced_names

# ফাংশন চালিয়ে ব্যালান্সড ডেটা লোড করুন
data, labels, image_names = load_and_balance_images(image_root_path, image_size)

# STEP 5: CSV ফাইল তৈরি
df = pd.DataFrame({
    "image_name": image_names,
    "label": labels
})
csv_path = "/content/fire_labels.csv"
df.to_csv(csv_path, index=False)

# STEP 6: CSV ফাইল ডাউনলোড করুন
files.download(csv_path)
print("📄 fire_labels.csv ডাউনলোডের জন্য প্রস্তুত।")

Saving Fire and Non-Fire satellite picture.zip to Fire and Non-Fire satellite picture.zip
✅ ডেটাসেট সফলভাবে আনজিপ করা হয়েছে!
✅ Fire/Non-Fire ফোল্ডার পাওয়া গেছে: /content/fire_dataset/Fire and Non-Fire satellite picture


Processing Non-Fire: 100%|██████████| 950/950 [00:01<00:00, 865.07it/s]


✅ ব্যালান্সড ডেটাসেট: Fire = 950, Non-Fire = 950


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📄 fire_labels.csv ডাউনলোডের জন্য প্রস্তুত।


In [ ]:
# ✅ STEP 1: প্রয়োজনীয় লাইব্রেরি ইমপোর্ট
from google.colab import files
import zipfile, os, shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# ✅ STEP 2: ফাইল আপলোড (ZIP এবং CSV)
print("⬆️ দয়া করে ZIP এবং CSV দুটি ফাইল আপলোড করুন...")
uploaded = files.upload()

# ✅ STEP 3: ফাইলের নাম বের করা
file_names = list(uploaded.keys())
csv_filename = [f for f in file_names if f.endswith(".csv")][0]
zip_filename = [f for f in file_names if f.endswith(".zip")][0]

# ✅ STEP 4: CSV লোড করা
df = pd.read_csv(csv_filename)
print("✅ CSV লোড হয়েছে! মোট লাইন:", len(df))

# ✅ STEP 5: ZIP ফাইল Extract করা
extract_dir = "/content/fire_dataset"
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("✅ ZIP ফাইল Extract হয়েছে!")

# ✅ STEP 6: Fire ও Non-Fire ফোল্ডার খোঁজা
def find_image_root(folder):
    for root, dirs, files in os.walk(folder):
        if 'Fire' in dirs and 'Non-Fire' in dirs:
            return root
    raise Exception("❌ Fire ও Non-Fire ফোল্ডার খুঁজে পাওয়া যায়নি!")

image_root_dir = find_image_root(extract_dir)
print("📁 Fire/Non-Fire ফোল্ডার:", image_root_dir)

# ✅ STEP 7: ফাইল ম্যাপ তৈরি (case-insensitive)
def build_file_map(root_dir):
    file_map = {}
    for cls in ['Fire', 'Non-Fire']:
        class_dir = os.path.join(root_dir, cls)
        for fname in os.listdir(class_dir):
            file_map[fname.lower()] = os.path.join(class_dir, fname)
    return file_map

file_map = build_file_map(image_root_dir)

# ✅ STEP 8: CSV অনুসারে split করা (Stratified split)
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)
print(f"📊 Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# ✅ STEP 9: গন্তব্য ফোল্ডার তৈরি
split_dir = "/content/fire_split_dataset"
for split in ['train', 'val', 'test']:
    for cls in ['Fire', 'Non-Fire']:
        os.makedirs(os.path.join(split_dir, split, cls), exist_ok=True)

# ✅ STEP 10: ফাইল কপি ফাংশন
def copy_images(dataframe, split_name):
    missing = 0
    copied = 0
    for _, row in dataframe.iterrows():
        label = 'Fire' if row['label'] == 1 else 'Non-Fire'
        img_name = row['image_name']
        img_name_lower = img_name.lower()

        if img_name_lower in file_map:
            src = file_map[img_name_lower]
            dst = os.path.join(split_dir, split_name, label, os.path.basename(src))
            shutil.copy(src, dst)
            copied += 1
        else:
            print(f"❌ ছবি পাওয়া যায়নি: {img_name}")
            missing += 1
    print(f"✅ {split_name} ➜ কপি হয়েছে: {copied}, মিসিং: {missing}")

# ✅ STEP 11: ছবি কপি করা
copy_images(train_df, 'train')
copy_images(val_df, 'val')
copy_images(test_df, 'test')

# ✅ STEP 12: ফাইনাল ZIP বানানো
shutil.make_archive("/content/fire_split_dataset", 'zip', split_dir)
files.download("/content/fire_split_dataset.zip")
print("📦 স্প্লিট ডেটাসেট ZIP তৈরি হয়েছে এবং ডাউনলোডের জন্য প্রস্তুত!")

⬆️ দয়া করে ZIP এবং CSV দুটি ফাইল আপলোড করুন...


Saving Fire and Non-Fire satellite picture.zip to Fire and Non-Fire satellite picture (1).zip
Saving fire_labels.csv to fire_labels (1).csv
✅ CSV লোড হয়েছে! মোট লাইন: 1900
✅ ZIP ফাইল Extract হয়েছে!
📁 Fire/Non-Fire ফোল্ডার: /content/fire_dataset/Fire and Non-Fire satellite picture
📊 Train: 1330 | Val: 285 | Test: 285
✅ train ➜ কপি হয়েছে: 1330, মিসিং: 0
✅ val ➜ কপি হয়েছে: 285, মিসিং: 0
✅ test ➜ কপি হয়েছে: 285, মিসিং: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📦 স্প্লিট ডেটাসেট ZIP তৈরি হয়েছে এবং ডাউনলোডের জন্য প্রস্তুত!


In [ ]:
# ✅ STEP 1: লাইব্রেরি ইমপোর্ট
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os

# ✅ STEP 2: ডেটাসেট লোকেশন
dataset_path = "/content/fire_split_dataset"

# ✅ STEP 3: ImageDataGenerator দিয়ে ইমেজ লোড
image_size = (250, 250)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_path, "train"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(dataset_path, "val"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(dataset_path, "test"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# ✅ STEP 4: ResNet50 লোড ও নতুন লেয়ার যুক্ত
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(250, 250, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# ✅ STEP 5: প্রথমে base_model ফ্রিজ করে নতুন লেয়ার ট্রেইন (10 ইপোক)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("📢 স্টেপ-১: শুধুমাত্র নতুন লেয়ার ট্রেইন হচ্ছে (10 epochs)")
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# ✅ STEP 6: এখন পুরো মডেল আনফ্রিজ করে ফাইন-টিউনিং (15 ইপোক)
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("📢 স্টেপ-২: পুরো মডেল ফাইন-টিউন হচ্ছে (15 epochs)")
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15
)

# ✅ STEP 7: টেস্ট সেটে Evaluate
loss, acc = model.evaluate(test_generator)
print(f"✅ টেস্ট সেটে একিউরেসি: {acc*100:.2f}%")

# ✅ STEP 8: মডেল সংরক্ষণ
model.save("/content/fire_detection_resnet50.h5")
print("✅ ResNet50 মডেল সেভ হয়েছে: fire_detection_resnet50.h5")

# ✅ STEP 9: মডেলটি লোকাল পিসিতে ডাউনলোড করা
from google.colab import files
files.download("/content/fire_detection_resnet50.h5")

Found 1330 images belonging to 2 classes.
Found 285 images belonging to 2 classes.
Found 285 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
📢 স্টেপ-১: শুধুমাত্র নতুন লেয়ার ট্রেইন হচ্ছে (10 epochs)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 422s 10s/step - accuracy: 0.5074 - loss: 0.8027 - val_accuracy: 0.4982 - val_loss: 0.6739
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 399s 10s/step - accuracy: 0.5921 - loss: 0.6717 - val_accuracy: 0.6526 - val_loss: 0.6336
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 398s 9s/step - accuracy: 0.7261 - loss: 0.6392 - val_accuracy: 0.7368 - val_loss: 0.6095
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 449s 10s/step - accuracy: 0.7397 - loss: 0.6200 - val_accuracy: 0.7544 - val_loss: 0.5803
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 403s 10s/step - accuracy: 0.7374 - loss: 0.6051 - val_accuracy: 0.8491 - val_loss: 0.5553
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 412s 10s/step - accuracy: 0.7985 - loss: 0.5722 - val_accuracy: 0.8456 - val_loss: 0.5355
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 397s 9s/step - accuracy: 0.7861 - loss: 0.5643 - val_accuracy: 0.8175 - val_loss: 0.5177
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 411s 10s/step - accuracy: 0.7825 - loss: 0.5571 - val_accuracy: 0.84

✅ টেস্ট সেটে একিউরেসি: 91.93%
✅ ResNet50 মডেল সেভ হয়েছে: fire_detection_resnet50.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow.keras.models import load_model

# সংরক্ষিত মডেল লোড করুন
model = load_model("/content/fire_detection_resnet50.h5")

In [ ]:
# ✅ STEP 1: প্রয়োজনীয় লাইব্রেরি
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab import files

# ✅ STEP 2: ট্রেইন করা মডেল লোড করুন
model_path = "/content/fire_detection_resnet50.h5"  # আপনার .h5 ফাইলের পথ
model = load_model(model_path)
print("✅ মডেল লোড হয়েছে সফলভাবে!")

# ✅ STEP 3: ছবি আপলোড করুন
uploaded = files.upload()

# ✅ STEP 4: সঠিক প্রেডিকশন ফাংশন (Fire=0, Non-Fire=1 হিসেবে ট্রেন হয়েছে)
def predict_fire(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("❌ ছবি লোড করা যায়নি:", image_path)
        return
    img = cv2.resize(img, (250, 250))               # রিসাইজ
    img = img.astype("float32") / 255.0             # নরমালাইজ
    img = np.expand_dims(img, axis=0)               # ব্যাচ ডাইমেনশন

    prediction = model.predict(img)[0][0]

    # 🔁 এখানে পরিবর্তন: Fire হলে prediction < 0.5
    label = "🔥 Fire" if prediction < 0.5 else "❄️ Non-Fire"
    print(f"✅ প্রেডিকশন: {label} (Confidence: {prediction:.2f})")

# ✅ STEP 5: সব আপলোড হওয়া ফাইলের উপর প্রেডিকশন চালানো
for fname in uploaded.keys():
    print(f"\n📷 ফাইল: {fname}")
    predict_fire(fname)

✅ মডেল লোড হয়েছে সফলভাবে!


Saving NonAgun2.jpg to NonAgun2.jpg

📷 ফাইল: NonAgun2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
✅ প্রেডিকশন: ❄️ Non-Fire (Confidence: 1.00)


In [ ]:

# STEP 1: প্রয়োজনীয় লাইব্রেরি ইন্সটল (Gradio যদি না থাকে)
!pip install -q gradio

# STEP 2: লাইব্রেরি ইমপোর্ট
import gradio as gr
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# STEP 3: মডেল লোড
model = load_model("/content/fire_detection_resnet50.h5")

# STEP 4: প্রেডিকশন ফাংশন (Fire=0, Non-Fire=1 ধরে)
def predict_fire_gradio(image):
    img = cv2.resize(image, (250, 250))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Ensure RGB
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]
    label = "🔥 Fire" if prediction < 0.5 else "❄️ Non-Fire"
    confidence = 1 - prediction if prediction < 0.5 else prediction
    return f"{label} (Confidence: {confidence:.2f})"

# STEP 5: Gradio ইন্টারফেস তৈরি
interface = gr.Interface(
    fn=predict_fire_gradio,
    inputs=gr.Image(type="numpy", label="আপনার ছবি আপলোড করুন"),
    outputs=gr.Textbox(label="প্রেডিকশন ফলাফল"),
    title="🔥 Fire Detection System",
    description="একটি স্যাটেলাইট বা যেকোনো ছবি আপলোড করুন, মডেল বলবে এটি Fire না Non-Fire।"
)

# STEP 6: Gradio অ্যাপ চালু
interface.launch(share=True)  # 'share=True' দিলে একটি public URL পাবেন

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f59e9217bac0e01386.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pickle



In [ ]:
pickle.dump(model,open("Resnet50.pkl", "wb"))